# Dashboard Semplificato ORTI / INTUR 2025

**Struttura:**
- RICAVI per BU: Hotel, Angelina, CVM, F&B, Spiaggia, Altri
- COSTI FISSI: lista specifica (affitti, canoni, leasing, utenze, manutenzioni, software)
- COSTI VARIABILI: acquisti + resto non fissi
- PERSONALE: da file PROSPETTO (RAR estratti)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import warnings
import re
warnings.filterwarnings('ignore')

DATA_DIR = Path('../data')
OUTPUT_DIR = Path('../output')
OUTPUT_DIR.mkdir(exist_ok=True)

# Mapping mesi
MESI_MAP = {
    '01_GENNAIO': ('Gennaio', 1), '02_FEBBRAIO': ('Febbraio', 2), '02_FEBBRAIIO': ('Febbraio', 2),
    '03_MARZO': ('Marzo', 3), '04_APRILE': ('Aprile', 4), '05_MAGGIO': ('Maggio', 5),
    '06_GIUGNO': ('Giugno', 6), '07_LUGLIO': ('Luglio', 7), '08_AGOSTO': ('Agosto', 8),
    '09_SETTEMBRE': ('Settembre', 9), '10_OTTOBRE': ('Ottobre', 10),
    '11_NOVEMBRE': ('Novembre', 11), '12_DICEMBRE': ('Dicembre', 12)
}

MESI_ORDINE = ['Gennaio', 'Febbraio', 'Marzo', 'Aprile', 'Maggio', 'Giugno',
               'Luglio', 'Agosto', 'Settembre', 'Ottobre', 'Novembre', 'Dicembre']

MESI_NUM = {m: i+1 for i, m in enumerate(MESI_ORDINE)}

In [ ]:
# === UTILITY ===

def clean_conto(val):
    """Pulisce codice conto"""
    if pd.isna(val):
        return ''
    if isinstance(val, datetime.timedelta):
        return ''
    s = str(val).replace('\xa0', '').strip()
    if s and s[0].isdigit() and '.' in s:
        return s
    return ''

def parse_importo(val):
    """Converte importo italiano a float"""
    if pd.isna(val) or val == '':
        return 0.0
    if isinstance(val, (int, float)):
        return float(val)
    s = str(val).replace('.', '').replace(',', '.')
    try:
        return float(s)
    except:
        return 0.0

def is_true_leaf(code, all_codes):
    """Codice foglia = nessun figlio"""
    prefix = code + '.'
    for other in all_codes:
        if other.startswith(prefix):
            return False
    return True

## 1. Classificazione Semplificata

In [ ]:
# === CLASSIFICAZIONE SEMPLIFICATA ===

# COSTI FISSI - lista esatta
COSTI_FISSI_CODES = [
    '65.11.01',       # Affitto azienda (ORTI->INTUR)
    '65.01.05',       # Canoni (tutti i sottocodi)
    '65.03.05',       # Leasing
    '65.05.',         # Noleggi
    '65.90.',         # Software/licenze
    '57.11.',         # Manutenzioni
    '57.09.',         # Utenze (telefono, reti, energia)
    '57.01.31',       # Energia elettrica
    '57.01.33',       # Gas
    '57.01.35',       # Acqua
    '63.',            # Ammortamenti
    '71.',            # Oneri diversi gestione
]

def classify_ricavo_bu(conto):
    """Classifica ricavo per BU"""
    if not (conto.startswith('47.') or conto.startswith('53.')):
        return None
    
    # HOTEL: 47.91.01.*
    if conto.startswith('47.91.01'):
        return 'HOTEL'
    
    # ANGELINA: 47.92.01.*
    if conto.startswith('47.92.01'):
        return 'ANGELINA'
    
    # CVM: 47.93.01.*
    if conto.startswith('47.93.01'):
        return 'CVM'
    
    # F&B: tutti i ricavi ristorazione
    if (conto.startswith('47.91.07') or conto.startswith('47.92.02') or 
        conto.startswith('47.93.02') or conto.startswith('47.94.07')):
        return 'F&B'
    
    # SPIAGGIA: 47.94.* escluso F&B
    if conto.startswith('47.94.') and not conto.startswith('47.94.07'):
        return 'SPIAGGIA'
    
    # ALTRI_RICAVI
    return 'ALTRI_RICAVI'


def is_costo_fisso(conto):
    """Verifica se conto è costo fisso"""
    for prefix in COSTI_FISSI_CODES:
        if conto.startswith(prefix):
            return True
    return False


def classify_costo(conto):
    """Classifica costo: FISSI, VARIABILI, PERSONALE"""
    if conto.startswith('47.') or conto.startswith('53.'):
        return None  # Ricavo
    
    # PERSONALE: 67.* + 61.*
    if conto.startswith('67.') or conto.startswith('61.'):
        return 'PERSONALE'
    
    # FISSI
    if is_costo_fisso(conto):
        return 'COSTI_FISSI'
    
    # VARIABILI: tutto il resto (55.*, 57.* non fissi, 59.*, ecc)
    return 'COSTI_VARIABILI'

## 2. Caricamento Dati Contabilità

In [ ]:
def load_mesepermese(filepath):
    """Carica dati mensili da mesepermese"""
    xls = pd.ExcelFile(filepath)
    monthly_data = {mese: {} for mese in MESI_ORDINE}
    
    for sheet_name in xls.sheet_names:
        if sheet_name not in MESI_MAP:
            continue
        
        mese_nome, _ = MESI_MAP[sheet_name]
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # Filtra Conto Economico
        df = df[df['Tipo conto e sezione'].str.contains('Conto Economico', na=False)].copy()
        df['Conto_clean'] = df['Conto'].apply(clean_conto)
        df['Importo_num'] = df['Importo'].apply(parse_importo)
        
        # Solo foglie
        all_codes = set(df[df['Conto_clean'] != '']['Conto_clean'].unique())
        true_leaves = {c for c in all_codes if is_true_leaf(c, all_codes)}
        
        for _, row in df[df['Conto_clean'].isin(true_leaves)].iterrows():
            conto = row['Conto_clean']
            val = row['Importo_num']
            if conto:
                if conto not in monthly_data[mese_nome]:
                    monthly_data[mese_nome][conto] = 0
                monthly_data[mese_nome][conto] += val
    
    return monthly_data

# Carica dati
print("Caricamento ORTI...")
orti_monthly = load_mesepermese(DATA_DIR / 'ORTI_mesepermese.xlsx')
print("Caricamento INTUR...")
intur_monthly = load_mesepermese(DATA_DIR / 'INTUR_mesepermese.xlsx')
print("Fatto!")

## 3. Analisi File Personale (PROSPETTO)

In [ ]:
# Esplora struttura file PROSPETTO
prospetto_orti_dir = DATA_DIR / 'personale' / 'PROSPETTO ORTI'
prospetto_intur_dir = DATA_DIR / 'personale' / 'PROSPETTO INTUR'

print("=" * 60)
print("FILE PROSPETTO ORTI:")
print("=" * 60)
for f in sorted(prospetto_orti_dir.glob('*.xlsx')):
    print(f"  {f.name}")

print("\n" + "=" * 60)
print("FILE PROSPETTO INTUR:")
print("=" * 60)
for f in sorted(prospetto_intur_dir.glob('*.xlsx')):
    print(f"  {f.name}")

In [ ]:
# Analizza struttura di un file PROSPETTO
sample_file = list(prospetto_orti_dir.glob('ORT_PC_03*.xlsx'))[0]
print(f"\nAnalisi: {sample_file.name}")
print("=" * 60)

df_sample = pd.read_excel(sample_file)
print(f"Colonne: {list(df_sample.columns)}")
print(f"\nRighe: {len(df_sample)}")
print("\nPrime 30 righe:")
print(df_sample.head(30).to_string())

In [ ]:
# Mostra tutte le voci uniche per capire quali sono retribuzioni
print("\nTutte le voci nel file:")
print("=" * 60)
# Cerca colonna descrizione
desc_col = None
for col in df_sample.columns:
    if 'descrizione' in str(col).lower() or 'voce' in str(col).lower():
        desc_col = col
        break

if desc_col:
    for v in df_sample[desc_col].dropna().unique():
        print(f"  - {v}")
else:
    # Prova prima colonna non numerica
    for col in df_sample.columns:
        if df_sample[col].dtype == 'object':
            print(f"\nColonna '{col}' - valori unici:")
            for v in df_sample[col].dropna().unique()[:30]:
                print(f"  - {v}")
            break

In [ ]:
# Definiamo le voci che compongono le RETRIBUZIONI
# (stipendi lordi + contributi azienda)

VOCI_RETRIBUZIONI = [
    'Retribuzioni',
    'Retrib. Stage',
    'Retrib.Collab.Coord.',
    'Contributi ASPI Inps',
    'Contributi Inps',
    'Accanton. 13ma',
    'Accanton. 14ma', 
    'Accanton. Ferie',
    'Accanton. R.O.L.',
    'Accanton. TFR mese',
    'Accanton. TFR retrib.differite',
    'Oneri previd. Accant. 13ma/14ma/Ferie/R.O.L.',
    'Premio Inail mese',
    'Premio Inail retrib.differite',
]

# Voci da ESCLUDERE (non sono costo azienda)
VOCI_ESCLUSE = [
    'Trasferte',
    'Rimborso spese',
    'Incentivo esodo',
    'Irpef',
    'Ritenute',
    'Competenze nette',
    'Fondo',
    'Debito',
    'Arrotondamento',
]

print("Voci considerate RETRIBUZIONI (costo azienda):")
for v in VOCI_RETRIBUZIONI:
    print(f"  + {v}")

In [ ]:
def extract_mese_from_filename(filename):
    """Estrae numero mese dal nome file (es: ORT_PC_03_2025.xlsx -> 3)"""
    match = re.search(r'_(\d{2})_2025', filename)
    if match:
        return int(match.group(1))
    return None

def load_prospetto_file(filepath):
    """Carica file prospetto e estrae retribuzioni"""
    df = pd.read_excel(filepath)
    
    # Trova colonne rilevanti
    desc_col = None
    dare_col = None
    avere_col = None
    tipo_col = None
    
    for col in df.columns:
        col_lower = str(col).lower()
        if 'descrizione' in col_lower:
            desc_col = col
        elif col_lower == 'dare':
            dare_col = col
        elif col_lower == 'avere':
            avere_col = col
        elif col_lower == 'tipo':
            tipo_col = col
    
    if not desc_col:
        # Fallback: cerca prima colonna stringa
        for col in df.columns:
            if df[col].dtype == 'object':
                desc_col = col
                break
    
    if not desc_col:
        return 0
    
    totale = 0
    for _, row in df.iterrows():
        desc = str(row.get(desc_col, '')).strip()
        tipo = str(row.get(tipo_col, '')).strip() if tipo_col else 'E'
        
        # Solo tipo E (Economico) - costi
        if tipo != 'E':
            continue
        
        # Verifica se è una voce retribuzione
        is_retrib = False
        for voce in VOCI_RETRIBUZIONI:
            if voce.lower() in desc.lower():
                is_retrib = True
                break
        
        # Escludi voci non pertinenti
        for excl in VOCI_ESCLUSE:
            if excl.lower() in desc.lower():
                is_retrib = False
                break
        
        if is_retrib:
            dare = parse_importo(row.get(dare_col, 0)) if dare_col else 0
            avere = parse_importo(row.get(avere_col, 0)) if avere_col else 0
            totale += dare - avere
    
    return totale

# Test su un file
test_val = load_prospetto_file(sample_file)
print(f"Test {sample_file.name}: {test_val:,.2f} EUR")

In [ ]:
def load_all_prospetto(directory, prefix=''):
    """Carica tutti i file prospetto e aggrega per mese"""
    monthly_totals = {m: 0 for m in MESI_ORDINE}
    
    for filepath in directory.glob('*.xlsx'):
        mese_num = extract_mese_from_filename(filepath.name)
        if mese_num and 1 <= mese_num <= 12:
            mese_nome = MESI_ORDINE[mese_num - 1]
            val = load_prospetto_file(filepath)
            monthly_totals[mese_nome] += val
            print(f"  {filepath.name}: mese {mese_num} ({mese_nome}) = {val:,.2f}")
    
    return monthly_totals

print("\n" + "=" * 60)
print("ORTI - Retribuzioni da PROSPETTO:")
print("=" * 60)
orti_personale = load_all_prospetto(prospetto_orti_dir)

print("\n" + "=" * 60)
print("INTUR - Retribuzioni da PROSPETTO:")
print("=" * 60)
intur_personale = load_all_prospetto(prospetto_intur_dir)

In [ ]:
# Riepilogo personale
print("\n" + "=" * 60)
print("RIEPILOGO PERSONALE (da PROSPETTO):")
print("=" * 60)
print(f"{'Mese':<12} {'ORTI':>15} {'INTUR':>15} {'TOTALE':>15}")
print("-" * 60)

tot_orti = 0
tot_intur = 0
for mese in MESI_ORDINE:
    o = orti_personale[mese]
    i = intur_personale[mese]
    tot_orti += o
    tot_intur += i
    if o > 0 or i > 0:
        print(f"{mese:<12} {o:>15,.2f} {i:>15,.2f} {o+i:>15,.2f}")

print("-" * 60)
print(f"{'TOTALE':<12} {tot_orti:>15,.2f} {tot_intur:>15,.2f} {tot_orti+tot_intur:>15,.2f}")

## 4. Build Dashboard Semplificato

In [ ]:
def build_dashboard_semplificato(monthly_data, personale_data, use_prospetto_personale=True):
    """Costruisce dashboard con struttura semplificata"""
    results = []
    
    for mese in MESI_ORDINE:
        data = monthly_data.get(mese, {})
        
        # RICAVI per BU
        ricavi = {'HOTEL': 0, 'ANGELINA': 0, 'CVM': 0, 'F&B': 0, 'SPIAGGIA': 0, 'ALTRI_RICAVI': 0}
        for conto, val in data.items():
            bu = classify_ricavo_bu(conto)
            if bu:
                ricavi[bu] += val
        
        # COSTI
        costi_fissi = 0
        costi_variabili = 0
        personale_contab = 0
        
        for conto, val in data.items():
            cat = classify_costo(conto)
            if cat == 'COSTI_FISSI':
                costi_fissi += val
            elif cat == 'COSTI_VARIABILI':
                costi_variabili += val
            elif cat == 'PERSONALE':
                personale_contab += val
        
        # Usa dati PROSPETTO per personale se disponibili
        if use_prospetto_personale and personale_data[mese] > 0:
            personale = personale_data[mese]
        else:
            personale = personale_contab
        
        tot_ricavi = sum(ricavi.values())
        tot_costi = costi_fissi + costi_variabili + personale
        ebitda = tot_ricavi - tot_costi
        
        results.append({
            'Mese': mese,
            'HOTEL': round(ricavi['HOTEL'], 2),
            'ANGELINA': round(ricavi['ANGELINA'], 2),
            'CVM': round(ricavi['CVM'], 2),
            'F&B': round(ricavi['F&B'], 2),
            'SPIAGGIA': round(ricavi['SPIAGGIA'], 2),
            'ALTRI_RICAVI': round(ricavi['ALTRI_RICAVI'], 2),
            'TOT_RICAVI': round(tot_ricavi, 2),
            'COSTI_FISSI': round(costi_fissi, 2),
            'COSTI_VARIABILI': round(costi_variabili, 2),
            'PERSONALE': round(personale, 2),
            'TOT_COSTI': round(tot_costi, 2),
            'EBITDA': round(ebitda, 2)
        })
    
    return pd.DataFrame(results)

In [ ]:
# Genera dashboard
dash_orti = build_dashboard_semplificato(orti_monthly, orti_personale)
dash_intur = build_dashboard_semplificato(intur_monthly, intur_personale)

print("ORTI - Dashboard Semplificato 2025:")
print("=" * 100)
display(dash_orti)

print("\nINTUR - Dashboard Semplificato 2025:")
print("=" * 100)
display(dash_intur)

In [ ]:
# Riepilogo annuale
print("\n" + "=" * 70)
print("RIEPILOGO ANNUALE")
print("=" * 70)

for name, df in [('ORTI', dash_orti), ('INTUR', dash_intur)]:
    print(f"\n{name}:")
    print(f"  Ricavi Totali:    {df['TOT_RICAVI'].sum():>15,.2f} EUR")
    print(f"  - Hotel:          {df['HOTEL'].sum():>15,.2f}")
    print(f"  - Angelina:       {df['ANGELINA'].sum():>15,.2f}")
    print(f"  - CVM:            {df['CVM'].sum():>15,.2f}")
    print(f"  - F&B:            {df['F&B'].sum():>15,.2f}")
    print(f"  - Spiaggia:       {df['SPIAGGIA'].sum():>15,.2f}")
    print(f"  - Altri:          {df['ALTRI_RICAVI'].sum():>15,.2f}")
    print(f"  Costi Totali:     {df['TOT_COSTI'].sum():>15,.2f} EUR")
    print(f"  - Fissi:          {df['COSTI_FISSI'].sum():>15,.2f}")
    print(f"  - Variabili:      {df['COSTI_VARIABILI'].sum():>15,.2f}")
    print(f"  - Personale:      {df['PERSONALE'].sum():>15,.2f}")
    print(f"  EBITDA:           {df['EBITDA'].sum():>15,.2f} EUR")

## 5. Export CSV per Google Sheets

In [ ]:
# Salva CSV
dash_orti.to_csv(OUTPUT_DIR / 'ORTI_dashboard_semplificato.csv', index=False)
dash_intur.to_csv(OUTPUT_DIR / 'INTUR_dashboard_semplificato.csv', index=False)

print("CSV esportati:")
print(f"  - {OUTPUT_DIR / 'ORTI_dashboard_semplificato.csv'}")
print(f"  - {OUTPUT_DIR / 'INTUR_dashboard_semplificato.csv'}")

# Mostra contenuto
print("\n" + "=" * 70)
print("Contenuto ORTI_dashboard_semplificato.csv:")
print("=" * 70)
print(dash_orti.to_csv(index=False))

In [ ]:
# Crea anche un foglio combinato
dash_orti['Azienda'] = 'ORTI'
dash_intur['Azienda'] = 'INTUR'

# Riordina colonne
cols = ['Azienda', 'Mese', 'HOTEL', 'ANGELINA', 'CVM', 'F&B', 'SPIAGGIA', 'ALTRI_RICAVI', 
        'TOT_RICAVI', 'COSTI_FISSI', 'COSTI_VARIABILI', 'PERSONALE', 'TOT_COSTI', 'EBITDA']

dash_combined = pd.concat([dash_orti[cols], dash_intur[cols]], ignore_index=True)
dash_combined.to_csv(OUTPUT_DIR / 'COMBINED_dashboard_2025.csv', index=False)

print("\nCSV combinato esportato:")
print(f"  - {OUTPUT_DIR / 'COMBINED_dashboard_2025.csv'}")